<a href="https://colab.research.google.com/github/ldhapple/moive_recommendation/blob/main/%EC%82%AC%EC%9A%A9%EC%9E%90_%EC%A7%91%EB%8B%A8%EB%B3%84_%EC%B6%94%EC%B2%9C%2C_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
import pandas as pd

base_src = '/content/data'
u_user_src = os.path.join(base_src,'u.user') #사용자 데이터
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep='|',
                    names=u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [26]:
u_item_src = os.path.join(base_src,'u.item') #영화 데이터
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src,
                     sep='|',
                     names=i_cols,
                     encoding = 'latin-1')
movies = movies.set_index('movie_id')
movies.head() 

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
u_data_src = os.path.join(base_src, 'u.data') #영화 평가 데이터
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src,
                      sep = '\t',
                      names = r_cols,
                      encoding = 'latin-1')
#ratings = ratings.set_index('user_id')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
#인기 제품 추천 방식

def recom_movie(n_items):
  movie_mean = ratings.groupby(['movie_id'])['rating'].mean() #rating에 대해 movie_id를 기준으로 평균
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items] #movie_mean을 평점이 높은 순으로 n_items 만큼 정렬
  recom_movies = movies.loc[movie_sort.index] #movie 데이터에서 movie_mean에서는 movie_name을 모르기 때문에 movie_sort를 이용해 이름 가져옴
  recommendations = recom_movies['title']
  return recommendations


recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [9]:
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

#정확도 계산 (RMSE 이용)
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating'] #각 사용자가 평가한 모든 영화의 평점 저장
  y_pred = movie_mean[ratings.loc[user]['movie_id']] #movie_id에 대한 예측값
  accuracy = RMSE(y_true, y_pred)
  rmse.append(accuracy)

print(np.mean(rmse))

0.7989582428029239


In [15]:
ratings = ratings.drop('timestamp', axis = 1)
movies = movies[['movie_id', 'title']]

In [18]:
#train set, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.25,
                                                    stratify = y) # x와 y데이터가 있고 train data를 75%, stratify = 데이터의 균등분배

#모델별 RMSE 계산
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 테스트 데이터 간의 user_id와 movie_id의 페어를 맞춰 튜플화
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs]) # 예측값
  y_true = np.array(x_test['rating']) # 테스트데이터의 실제 값
  return RMSE(y_true, y_pred)

# best_seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id, movie_id):
  try:
    rating = train_mean[movie_id] # 전체 데이터로 했을 때는 어떤 데이터를 넣어도 해당하는 영화가 있음. 하지만 분할했기 때문에 try-except

  except:
    rating = 3.0

  return rating

score(best_seller) # train data와 test data로 나누었기 때문에 성능저하 발견

1.0224593667926531

In [19]:
#성별에 따른 예측값 계산
merged_ratings = pd.merge(x_train, users)

users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean() #id, sex, rating을 가져오고 id,sex 기준 rating 평균
g_mean

movie_id  sex
1         F      3.887500
          M      3.896414
2         F      3.214286
          M      3.132530
3         F      2.857143
                   ...   
1676      M      2.000000
1678      M      1.000000
1680      M      2.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3027, dtype: float64

In [20]:
rating_matrix = x_train.pivot(index = 'user_id',
                              columns = 'movie_id',
                              values = 'rating')

# Gender 기준 추천

def cf_gender(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    gender = users.loc[user_id]['sex']
    if gender in g_mean[movie_id].index:
      gender_rating = g_mean[movie_id][gender]
    else:
      gender_rating = 3.0 # 영화가 성별의 평균을 가지고 있지 않을 경우
  else:
    gender_rating = 3.0 # movie_id가 matrix에 없을 경우
  return gender_rating

score(cf_gender) # 시간도 걸렸고 성능개선도 크게 이루어지지 않음.


1.0332651344210437

In [21]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0) # 원본 손상 방지
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index = rating_matrix.index,
                               columns = rating_matrix.index)

# 주어진 영화의 가중평균 rating을 계산하는 함수
def CF_simple(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    sim_scores = user_similarity[user_id].copy() # 주어진 사용자와 다른 사용자간 유사도를 추출
    movie_ratings = rating_matrix[movie_id].copy() # 주어진 영화에 대한 다른 사용자들의 평가
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index # 평가하지 않은 사람들의 index 추출
    movie_ratings = movie_ratings.dropna() # 평가하지 않은 부분 제외
    sim_scores = sim_scores.drop(none_rating_idx) # sim_scores 에서도 평가하지 않은 부분 제외
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() # 유사도와 영화에 대한 평점을 가중평균을 해서 sum으로 나눔
  else:
    mean_rating = 3.0
  return mean_rating

# 정확도 계산
score(CF_simple) # 단순 CF알고리즘이지만 단순 집단 분류 추천 방식보다 더 나은 성능을 보여줌.


1.0170807150266636

In [22]:
def score(model, neighbor_size = 0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

In [23]:
#KNN 방식
def CF_knn(user_id, movie_id, neighbor_size = 0):
  if movie_id in rating_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores)) #sim_score보다 neighbor_size가 클 수 있기 때문에 크기를 맞춰줌.
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores) #sim_scores를 오름차순으로 index값을 추출
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:] #neighbor_size 만큼 추출
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() # 행렬 곱

      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0
  
  return mean_rating

score(CF_knn, neighbor_size = 30) # 이웃의 크기를 고려하지 않는 단순 CF방법보다 더 나은 성능 도출.

1.0095113356460417

In [24]:
def recom_movie(user_id, n_items, neighbor_size = 30):
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0 # 해당 영화를 이미 평가한 경우는 제외.
    
    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(user_id = 777, n_items = 5, neighbor_size = 30)

movie_id
1189                              That Old Feeling (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1467                                     Cure, The (1995)
1656                                        Target (1995)
169                                Cinema Paradiso (1988)
Name: title, dtype: object

In [ ]:
# 최적의 이웃 크기 결정 (40일때 가장 나은 성능)
for neighbor_size in [10,20,30,40,50,60]:
  print('Neighbor size = %d : RMSE = %.4f' %(neighbor_size, score(CF_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0267
Neighbor size = 20 : RMSE = 1.0116
Neighbor size = 30 : RMSE = 1.0081
Neighbor size = 40 : RMSE = 1.0083
Neighbor size = 50 : RMSE = 1.0089
Neighbor size = 60 : RMSE = 1.0098


In [31]:
# 사용자 평가 경향을 고려한 함수
rating_mean = rating_matrix.mean(axis = 1)
rating_bias = (rating_matrix.T - rating_mean).T

def CF_knn_bias(user_id, movie_id, neighbor_size = 0):
  if movie_id in rating_bias.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]
    
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]

      else:
        prediction = rating_mean[user_id]
  else:
    prediction = rating_mean[user_id]

  return prediction

score(CF_knn_bias, neighbor_size = 30) # 사용자 평가 경향을 고려했을 때 더 나은 성능

0.9423894877261847

In [33]:
# 신뢰도 가중
rating_binary_1 = np.array(rating_matrix > 0).astype(float) # null값 제외 0 초과 값에 대해 true가 나오고, 이 외는 false를 배열로 만든다. float로 타입변경 하면 true = 1, flase = 0
rating_binary_2 = rating_binary_1.T

counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts,
                      index = rating_matrix.index,
                      columns = rating_matrix.index).fillna(0)

counts # 각 번호끼리 공통적으로 평가한 영화의 개수

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,204.0,10.0,5.0,3.0,46.0,53.0,91.0,19.0,4.0,38.0,...,45.0,9.0,23.0,8.0,19.0,10.0,25.0,5.0,15.0,45.0
2,10.0,46.0,4.0,2.0,1.0,18.0,7.0,0.0,3.0,8.0,...,6.0,8.0,16.0,7.0,12.0,5.0,10.0,2.0,6.0,5.0
3,5.0,4.0,41.0,8.0,0.0,5.0,6.0,4.0,0.0,5.0,...,1.0,1.0,11.0,3.0,5.0,0.0,10.0,1.0,4.0,1.0
4,3.0,2.0,8.0,18.0,0.0,3.0,5.0,2.0,0.0,2.0,...,1.0,0.0,3.0,4.0,5.0,1.0,7.0,2.0,4.0,1.0
5,46.0,1.0,0.0,0.0,131.0,28.0,66.0,11.0,2.0,16.0,...,38.0,2.0,6.0,2.0,8.0,0.0,16.0,3.0,9.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,10.0,5.0,0.0,1.0,0.0,6.0,10.0,1.0,0.0,3.0,...,4.0,14.0,13.0,6.0,19.0,37.0,2.0,3.0,3.0,7.0
940,25.0,10.0,10.0,7.0,16.0,34.0,37.0,9.0,2.0,29.0,...,24.0,3.0,16.0,7.0,11.0,2.0,80.0,4.0,13.0,12.0
941,5.0,2.0,1.0,2.0,3.0,4.0,5.0,3.0,2.0,2.0,...,2.0,2.0,10.0,4.0,7.0,3.0,4.0,16.0,1.0,2.0


In [35]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size = 0):
  if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()

    no_rating = movie_ratings.isnull() 
    common_counts = counts[user_id] # 선택한 user의 공통 평가 개수
    low_significance = common_counts < SIG_LEVEL # SIG_LEVEL 보다 공통 평가 영화 수가 적으면 true 반환
    none_rating_idx = movie_ratings[no_rating | low_significance].index # 평가 X index와 low_sig index 추출

    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]

    else:
      if len(sim_scores) > MIN_RATINGS:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]

      else:
        prediction = rating_mean[user_id]

  else:
    prediction = rating_mean[user_id]
  
  return prediction
        
SIG_LEVEL = 3 # 공통평가 영화수
MIN_RATINGS = 3 # 최소 사용자 수
score(CF_knn_bias_sig, 30) # 사용자 평가 경향때보다 일단 향상은 없음.

0.942741657230444

In [37]:
# 아이템 기반 CF

rating_matrix_t = np.transpose(rating_matrix) # 아이템간의 유사도를 측정해야 하기 때문에 transpose 시킴

matrix_dummy = rating_matrix_t.copy().fillna(0)

item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity,
                               index = rating_matrix_t.index,
                               columns = rating_matrix_t.index)

item_similarity # movie_id 간의 유사도

movie_id,1,2,3,4,5,6,7,8,9,10,...,1670,1671,1672,1674,1675,1676,1678,1680,1681,1682
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.263915,0.292047,0.350339,0.191177,0.049678,0.450177,0.353524,0.386227,0.198698,...,0.0,0.0,0.038818,0.0,0.0,0.0,0.0,0.0,0.054898,0.054898
2,0.263915,1.000000,0.175879,0.375836,0.241796,0.054859,0.266242,0.201278,0.155604,0.102532,...,0.0,0.0,0.065372,0.0,0.0,0.0,0.0,0.0,0.092450,0.092450
3,0.292047,0.175879,1.000000,0.284315,0.129337,0.098683,0.284834,0.145345,0.201025,0.125755,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.113389
4,0.350339,0.375836,0.284315,1.000000,0.302316,0.081877,0.393275,0.307921,0.274567,0.240152,...,0.0,0.0,0.047210,0.0,0.0,0.0,0.0,0.0,0.066766,0.089021
5,0.191177,0.241796,0.129337,0.302316,1.000000,0.047174,0.271022,0.166315,0.180930,0.055948,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077296,0.000000,0.089924,...,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.000000,0.000000
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000


In [40]:
def CF_IBCF(user_id, movie_id):
  if movie_id in item_similarity.columns:
    sim_scores = item_similarity[movie_id]
    user_rating = rating_matrix_t[user_id]
    none_rating_idx = user_rating[user_rating.isnull()].index
    user_rating = user_rating.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()

  else:
    mean_rating = 3.0

  return mean_rating

# def score(model):
#   id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 테스트 데이터 간의 user_id와 movie_id의 페어를 맞춰 튜플화
#   y_pred = np.array([model(user,movie) for (user,movie) in id_pairs]) # 예측값
#   y_true = np.array(x_test['rating']) # 테스트데이터의 실제 값
#   return RMSE(y_true, y_pred)

score(CF_IBCF)

1.0196922410452316